In [ ]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add parent directory to path
sys.path.insert(0, str(Path.cwd().parent))

from app.comet.mcp_client import MCPClient
from app.comet.prompt_manager import PromptManager

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

## 1. Load Backtest Results

In [ ]:
# Load backtest CSV files
backtest_dir = Path('../backtest_results')

# Example: Load Volume Rocket results
volume_rocket_df = pd.read_csv('../volume_rocket_results.csv')

print(f"Loaded {len(volume_rocket_df)} trades")
volume_rocket_df.head()

## 2. Performance Metrics

In [ ]:
# Calculate key metrics
total_trades = len(volume_rocket_df)
winning_trades = len(volume_rocket_df[volume_rocket_df['pnl'] > 0])
losing_trades = len(volume_rocket_df[volume_rocket_df['pnl'] < 0])
win_rate = (winning_trades / total_trades * 100) if total_trades > 0 else 0

total_pnl = volume_rocket_df['pnl'].sum()
avg_win = volume_rocket_df[volume_rocket_df['pnl'] > 0]['pnl'].mean()
avg_loss = volume_rocket_df[volume_rocket_df['pnl'] < 0]['pnl'].mean()
profit_factor = abs(avg_win / avg_loss) if avg_loss != 0 else 0

print(f"""\n=== Performance Summary ===
Total Trades: {total_trades}
Winning Trades: {winning_trades}
Losing Trades: {losing_trades}
Win Rate: {win_rate:.2f}%
\nTotal P&L: ₹{total_pnl:,.2f}
Avg Win: ₹{avg_win:,.2f}
Avg Loss: ₹{avg_loss:,.2f}
Profit Factor: {profit_factor:.2f}
""")

## 3. Visualizations

In [ ]:
# P&L Distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. P&L Histogram
axes[0, 0].hist(volume_rocket_df['pnl'], bins=50, edgecolor='black')
axes[0, 0].set_title('P&L Distribution')
axes[0, 0].set_xlabel('P&L (₹)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(0, color='red', linestyle='--', label='Breakeven')
axes[0, 0].legend()

# 2. Cumulative P&L
volume_rocket_df['cumulative_pnl'] = volume_rocket_df['pnl'].cumsum()
axes[0, 1].plot(volume_rocket_df['cumulative_pnl'])
axes[0, 1].set_title('Cumulative P&L Over Time')
axes[0, 1].set_xlabel('Trade Number')
axes[0, 1].set_ylabel('Cumulative P&L (₹)')
axes[0, 1].grid(True)

# 3. Win/Loss Ratio
win_loss_data = [winning_trades, losing_trades]
axes[1, 0].pie(win_loss_data, labels=['Wins', 'Losses'], autopct='%1.1f%%', startangle=90)
axes[1, 0].set_title('Win/Loss Ratio')

# 4. P&L by Trade Type (if available)
if 'signal_type' in volume_rocket_df.columns:
    pnl_by_type = volume_rocket_df.groupby('signal_type')['pnl'].sum()
    axes[1, 1].bar(pnl_by_type.index, pnl_by_type.values)
    axes[1, 1].set_title('P&L by Signal Type')
    axes[1, 1].set_xlabel('Signal Type')
    axes[1, 1].set_ylabel('Total P&L (₹)')
    axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 4. Comet AI Analysis - Best Trades

In [ ]:
# Initialize Comet AI
comet = MCPClient()

# Get top 5 winning trades
top_winners = volume_rocket_df.nlargest(5, 'pnl')

print("\n=== Analyzing Top 5 Winning Trades with Comet AI ===")
for idx, trade in top_winners.iterrows():
    print(f"\nTrade #{idx}: {trade.get('symbol', 'N/A')} - P&L: ₹{trade['pnl']:,.2f}")
    
    # Simple analysis query
    query = f"""Analyze this winning trade and identify success factors:
    Symbol: {trade.get('symbol', 'N/A')}
    Entry: {trade.get('entry_price', 'N/A')}
    Exit: {trade.get('exit_price', 'N/A')}
    P&L: ₹{trade['pnl']:,.2f}
    Duration: {trade.get('duration', 'N/A')}
    
    What made this trade successful? List 3 key factors.
    """
    
    try:
        response = comet.query(query)
        print(f"\nComet Analysis: {response.get('content', 'No response')}")
    except Exception as e:
        print(f"Error analyzing trade: {e}")

## 5. Comet AI Analysis - Losing Trades

In [ ]:
# Get top 5 losing trades
top_losers = volume_rocket_df.nsmallest(5, 'pnl')

print("\n=== Analyzing Top 5 Losing Trades with Comet AI ===")
for idx, trade in top_losers.iterrows():
    print(f"\nTrade #{idx}: {trade.get('symbol', 'N/A')} - P&L: ₹{trade['pnl']:,.2f}")
    
    query = f"""Analyze this losing trade and identify failure points:
    Symbol: {trade.get('symbol', 'N/A')}
    Entry: {trade.get('entry_price', 'N/A')}
    Exit: {trade.get('exit_price', 'N/A')}
    P&L: ₹{trade['pnl']:,.2f}
    Duration: {trade.get('duration', 'N/A')}
    
    Why did this trade fail? List 3 key issues and how to avoid them.
    """
    
    try:
        response = comet.query(query)
        print(f"\nComet Analysis: {response.get('content', 'No response')}")
    except Exception as e:
        print(f"Error analyzing trade: {e}")

## 6. Strategy Optimization Suggestions

In [ ]:
# Get comprehensive strategy analysis from Comet
strategy_query = f"""Based on this backtest data for a trading strategy:

- Total Trades: {total_trades}
- Win Rate: {win_rate:.2f}%
- Average Win: ₹{avg_win:,.2f}
- Average Loss: ₹{avg_loss:,.2f}
- Profit Factor: {profit_factor:.2f}
- Total P&L: ₹{total_pnl:,.2f}

Provide 5 specific recommendations to improve this strategy's performance.
Focus on actionable improvements that can be coded/backtested.
"""

try:
    optimization = comet.query(strategy_query)
    print("\n=== Strategy Optimization Recommendations ===")
    print(optimization.get('content', 'No recommendations'))
except Exception as e:
    print(f"Error getting optimization suggestions: {e}")

## 7. Export Analysis Report

In [ ]:
# Create a summary report
report = f"""\n# Backtest Analysis Report
## Generated: {pd.Timestamp.now()}

### Performance Metrics
- Total Trades: {total_trades}
- Win Rate: {win_rate:.2f}%
- Total P&L: ₹{total_pnl:,.2f}
- Average Win: ₹{avg_win:,.2f}
- Average Loss: ₹{avg_loss:,.2f}
- Profit Factor: {profit_factor:.2f}

### Key Insights
(Add insights from Comet AI analysis above)

### Recommendations
(Add optimization recommendations from above)
"""

print(report)

# Optionally save to file
# with open('../backtest_results/analysis_report.md', 'w') as f:
#     f.write(report)